In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
url = "https://mars.nasa.gov/news"
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_twtr = "https://twitter.com/MarsWxReport?lang=en"
url_table = "https://space-facts.com/mars/"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [4]:
browser = init_browser()
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
#news_text = soup.find('div', class_='list_text')
#new_parsing = news_text.find("div", class_="content_title")
news_title = soup.find('a', target="_self")

In [5]:
browser = init_browser()
browser.visit(url_img)
html = browser.html
soup = bs(html, "html.parser")
mars_img = browser.click_link_by_partial_text("FULL IMAGE")

C:\Users\jacob\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [6]:
browser = init_browser()
browser.visit(url_twtr)
html = browser.html
soup = bs(html, "html.parser")
mars_weather = soup.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")

In [7]:
mars_table = pd.read_html(url_table)

In [8]:
browser = init_browser()
browser.visit(url_hemispheres)
html = browser.html
soup = bs(html, "html.parser")
hemi_list = soup.find('div', class_ = 'collapsible results')
mars_hemis = hemi_list.find_all('div', class_='item')
mars_hemis
hemi_imgs = []
for hemi in mars_hemis:
    i = hemi.find('img')
    hemi_imgs.append(i)

In [9]:
listings = {"News Title": news_title,
           "Mars Image": mars_img,
           "Mars Weather": mars_weather,
           "Mars Table": mars_table,
           "Hemisphere 1": hemi_imgs[0],
           "Hemisphere 2": hemi_imgs[1],
           "Hemisphere 3": hemi_imgs[2],
           "Hemisphere 4": hemi_imgs[3]}